In [1]:
#Importamos la libreria pandas
import pandas as pd 
#Importamos la libreria seaborn
import seaborn as sns
#Importamos la libreria numpy
import numpy as np
#Importamos LabelEncoder desde la libreria sklearn
from sklearn.preprocessing import LabelEncoder
#Importamos train_test_split desde la libreria sklearn
from sklearn.model_selection import train_test_split
#Importamos MinMaxScaler desde la libreria sklearn
from sklearn.preprocessing import MinMaxScaler
#Importamos la libreria matplotlib
import matplotlib.pyplot as plt
#Importamos GaussianNB desde la libreria sklearn
from sklearn.naive_bayes import GaussianNB
#Importamos Metric desde la libreria sklearn
from sklearn import metrics
#Importamos la libreria collections
import collections
#Importamos hiden markov model desde la libreria hmmlearn
#from hmmlearn import hmm
#Importamos confusion_matrix desde la libreria sklearn
from sklearn.metrics import confusion_matrix
# Importamos acurracy_score desde la libreria sklearn
from sklearn.metrics import accuracy_score
#Importamos recall_score desde la libreria sklearn
from sklearn.metrics import recall_score
#Importamos precision_score desde la libreria sklearn
from sklearn.metrics import precision_score
#Importamos f1_score desde la libreria sklearn
from sklearn.metrics import f1_score
#Importamos la libreria de fuzzy logic
#from fuzzywuzzy import process
#Importamos StandardScaler de la libreria sklearn
from sklearn.preprocessing import StandardScaler

In [2]:
# Establecemos el enlace de conexión del dataset
url = 'https://raw.githubusercontent.com/sfbarragan/AlgoritmosProbabilisticos/master/neo.csv'
# Cargamos el dataset
df = pd.read_csv(url)
# Mostramos los datos del dataset
df.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249224,5.483974e+07,Earth,False,16.73,False
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726663,6.143813e+07,Earth,False,20.00,True
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692129,4.979872e+07,Earth,False,17.83,False
3,3596030,(2012 BV13),0.096506,0.215794,24764.303138,2.543497e+07,Earth,False,22.20,False
4,3667127,(2014 GE35),0.255009,0.570217,42737.733765,4.627557e+07,Earth,False,20.09,True


# Descripción del dataset

El dataset contiene información de mas de 90000 asteroides. Dicho dataset contiene 10 atributos, incluida una característica de clase (objetivo) que indica si el asteroide es o no peligroso (True/False). El significado comercial de cada columna (atributo) en los datos es el siguiente:

**Variable objetivo**<br>
hazardous: Clasificación del asteroide como peligroso o no (True/False).<br>

**Variables de entrada**<br>
id: Identificador único de cada asteroide <br>
name: Nombre del asteroide dado por la NASA<br>
est_diameter_min: Diámetro mínimo estimado en kilómetros<br>
est_diameter_max: Diámetro Máximo Estimado en Kilómetros<br>
relative_velocity: Velocidad relativa a la Tierra<br>
miss_distance: Distancia en Kilómetros perdidos<br>
orbiting_body: Planeta que órbita el asteroide<br>
sentry_object: Sistema automatizado de monitoreo de colisiones<br>
absolute_magnitude: Describe la luminosidad intrínseca<br>


# Problema a resolver

Este dataset contiene varios parámetros/características según los cuales un asteroide en particular que ya está clasificado como el objeto más cercano a la Tierra. En base a dichos parámetros se desea predecir si el asteroide puede o no ser peligroso.

Como conocemos que no podemos procesar esa cantidad de datos, hemos optado por realizar un sampling de los datos tomado como muestra un total de 2500 datos para entrenar el modelo. Donde la variable objetivo se denomina hazardous. Existen 9 variables de entrada llamadas: id, name, est_diameter_min, est_diameter_max, relative_velocity, miss_distance, orbiting_body, sentry_object y absolute_magnitude.

In [3]:
# Obtenemos una muestra del dataset
data_muestra = df.sample(n=2500)
# Mostramos los datos de la muestra
data_muestra.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous
9194,3792462,(2017 YD),0.021113,0.047211,41173.567906,4.030595e+07,Earth,False,25.50,False
90591,3994621,(2020 DT2),0.008027,0.017949,49475.956157,5.478926e+07,Earth,False,27.60,False
80085,3789485,(2017 WV1),0.005553,0.012418,70409.590407,6.269029e+07,Earth,False,28.40,False
83450,54101718,(2020 YC3),0.244655,0.547066,67987.904735,5.921640e+07,Earth,False,20.18,True
46011,3578688,(2011 SR12),0.319562,0.714562,85809.835601,3.524894e+07,Earth,False,19.60,False


In [4]:
# Contamos los registros del dataset
data_muestra.count()

id                    2500
name                  2500
est_diameter_min      2500
est_diameter_max      2500
relative_velocity     2500
miss_distance         2500
orbiting_body         2500
sentry_object         2500
absolute_magnitude    2500
hazardous             2500
dtype: int64

# Estadisticas generales


In [5]:
#Se hace una descripcion para ver como esta conformado el dataset 
data_muestra.describe()

,id,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude
count,2.500000e+03,2500.000000,2500.000000,2500.000000,2.500000e+03,2500.000000
mean,1.438517e+07,0.130540,0.291897,48797.034405,3.743882e+07,23.456856
std,2.087742e+07,0.249019,0.556824,25045.956413,2.248432e+07,2.879889
min,2.001943e+06,0.001058,0.002366,2601.909336,1.430546e+04,14.130000
25%,3.467247e+06,0.020163,0.045086,29554.914035,1.782002e+07,21.300000
50%,3.746470e+06,0.048368,0.108153,46117.203929,3.864467e+07,23.700000
75%,3.892174e+06,0.146068,0.326618,64275.146198,5.704828e+07,25.600000
max,5.427276e+07,3.967848,8.872377,151528.295999,7.477560e+07,32.000000


# Describir resultados estadísticos


Podemos observar una descripción general de los datos de muestra del dataset, observamos la media, varinza, valores minimos y valores maximos. Esto nos da un idea de como esta organizada la data del dataset.

# Descripción de la data


In [6]:
# Mostramos la estructura de los atributos del dataset
data_muestra.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 9194 to 81448
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  2500 non-null   int64  
 1   name                2500 non-null   object 
 2   est_diameter_min    2500 non-null   float64
 3   est_diameter_max    2500 non-null   float64
 4   relative_velocity   2500 non-null   float64
 5   miss_distance       2500 non-null   float64
 6   orbiting_body       2500 non-null   object 
 7   sentry_object       2500 non-null   bool   
 8   absolute_magnitude  2500 non-null   float64
 9   hazardous           2500 non-null   bool   
dtypes: bool(2), float64(5), int64(1), object(2)
memory usage: 180.7+ KB


# Limpieza del dataset

In [7]:
#Verificamos la existencia de datos nulos
data_muestra.isnull().sum()

id                    0
name                  0
est_diameter_min      0
est_diameter_max      0
relative_velocity     0
miss_distance         0
orbiting_body         0
sentry_object         0
absolute_magnitude    0
hazardous             0
dtype: int64

In [8]:
#Verificamos la existencia  de datos vacios (longitud = 0)
data_muestra.isna().sum()

id                    0
name                  0
est_diameter_min      0
est_diameter_max      0
relative_velocity     0
miss_distance         0
orbiting_body         0
sentry_object         0
absolute_magnitude    0
hazardous             0
dtype: int64

In [9]:
#Eliminamos las columnas que no utilizaremos
data_muestra = data_muestra.drop(columns=['id','name'])